# Question 1 [25 Marks]

1. In this part, you are required to implement and evaluate the Apriori-based algorithm for frequent itemsets mining. **You are not allowed to use any libarary.** You are free to consult any online resources but be careful while implementing the solution. 

2. The program should be executable with 3 parameters: the name of the input dataset file, the threshold of minimum support count, and the name of the output file.  The minimum support count should be an integer.  An itemset is frequent if its support count is larger or equal to this threshold. 

3. There are 5 bonus points for storing the identified frequent itemsets in Excel file. 

4. The program should output a file that contains all the frequent itemsets together with their support.  The output file (sample output) should have the following format: each line contains a single frequent itemset as a list of items separated by whitespace.  At the end of the line, its support is printed between a pair of parenthesis.  For example: 1 2 3 (5) represents an itemset containing items 1, 2 and 3 with a support count of 5.


In [74]:
import pandas as pd
import numpy as np
import csv

In [92]:
#file reading from dataset.csv
#you are free to use any data loader i-e you can use pandas, np or any method you want.
dataSet = pd.read_csv('dataset.csv', sep='delimiter', header=None)
dataSet

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,0
0,"39,120,124,205,401,581,704,814,825,834"
1,"35,249,674,712,733,759,854,950"
2,"39,422,449,704,825,857,895,937,954,964"
3,"15,229,262,283,294,352,381,708,738,766,853,883..."
4,"26,104,143,320,569,620,798"
...,...
16016,"12,196,347,350,758,766,782,789,804,820,823,829..."
16017,"69,126,130,279,361,362,878,914"
16018,"6,27,31,32,440,450,472,494,517,647,818,883,893..."
16019,"290,422,440,449,450,574,752,895,919,937"


In [93]:
dataSet_l= [i[0].split(",") for i in dataSet.values]

### Identify unique items [5 Marks]
From the list of transactions, identify and list all unique items in the dataset.


In [94]:
unique_l=[]
for lis in dataSet_l:
    for x in lis:
        if x not in unique_l:
            unique_l.append(x)

In [95]:
print('Total unique values',len(unique_l))

Total unique values 869


In [111]:
for x in unique_l:
  print(x,end=',')

0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,58,59,62,64,66,67,68,69,70,71,72,73,74,75,78,80,81,82,83,85,86,88,89,90,91,93,94,95,96,97,98,100,101,102,103,104,105,108,110,111,112,113,114,115,116,117,118,120,121,122,123,124,125,126,127,128,129,130,131,132,135,136,137,138,139,140,141,142,143,144,145,146,147,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,185,186,187,188,189,190,191,192,193,194,196,197,198,199,201,202,203,204,205,206,207,208,209,210,212,213,214,215,216,217,218,220,222,224,225,226,227,228,229,230,231,234,236,237,239,240,241,242,244,245,246,247,248,249,252,254,255,256,257,258,259,261,262,263,264,265,266,267,268,269,270,272,273,274,275,276,277,278,279,280,281,282,283,285,286,287,289,290,292,293,294,295,296,297,298,299,300,301,303,304,305,306,307,308,309,310,311,313,314,315,316,317

In [97]:
unique_l=sorted(unique_l, key=int)

## Implementation [20 Marks]

Test your implementation on the dataset dataset.csv and measure number of frequent itemsets with various minimum support values.  The test dataset is a synthetic dataset that contains approx 16,000 transactions.

In [99]:
def createC1(dataSet, C):
    '''
    Add your code here. Function would receive data set and return C1 set.
    '''
    for i in range(len(unique_l)):
        unique_l[i] = str(unique_l[i])
    C.update({1:[[x] for x in unique_l]})
    return C

In [100]:
#Looking for k-frequent itemsets
def selectLk(dataSet,Ck,minSupport, already_removed): #dataSet raw data set
    '''
    Add your code here. Function would receive data set, Ck and minimum support. Funtion will return pruined items from Ck.
    '''
    length = len(already_removed.keys())
    L = []
    sup = []
    toberemoved = []
    numDataSet = len(dataSet)
    for i in range(len(Ck)):
        hdb = True
        if length > 0:
            for item in already_removed[length]:
                if set(item).issubset(set(Ck[i])):
                    hdb = False
                    break
        if hdb:
            count=0
            for j in range(len(dataSet)):
                if set(Ck[i]).issubset(set(dataSet[j])):
                    count+=1
            if (count/len(dataSet)) >= minSupport:
                L.append(Ck[i])
                sup.append(count)
            else:
                toberemoved.append(Ck[i])
    return L, sup, toberemoved

In [101]:
def createCk(Lk): # Lk: Frequent itemsets containing k items
    '''
    Add your code here. Function would receive Lk frequent items set and it's count. Function should return Ck.
    '''
    C=[]
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            item=[]
            Lk[i].sort(key=lambda x: unique_l.index(x))
            Lk[j].sort(key=lambda x: unique_l.index(x))
            for z in range(len(Lk[i])-1):
              if Lk[i][z]!=Lk[j][z]:
                break
            if unique_l.index(Lk[i][-1])<unique_l.index(Lk[j][-1]):
              item = Lk[i]+[Lk[j][-1]]
            if len(item)>0:
                C.append(item)
    return C

In [102]:
#Apriori algorithm
def apriori(dataSet,minSupport):
    comb=1
    removed={comb:[]}
    #create C1
    C={}
    C=createC1(dataSet, C)
    #select L1
    L={}
    supc = {}
    l,sup, remove=selectLk(dataSet,C[comb], minSupport, removed)
    L.update({comb: l})
    removed.update({comb: remove})
    supc.update({comb: sup})
    #Iterative create and select Ck & Lk iteratively.
    comb += 1
    convergence = False
    while convergence == False:
        C.update({comb: createCk(L[comb-1])})
        l,sup, remove=selectLk(dataSet,C[comb], minSupport, removed)
        L.update({comb: l})
        removed.update({comb: remove})
        supc.update({comb: sup})
        if len(L[comb])==0:
            convergence=True
        comb += 1
    return L, supc

In [112]:
# Do not change this cell
L = apriori(dataSet_l ,0.02)
for x in L:
  print(x)

{1: [['6'], ['8'], ['12'], ['21'], ['27'], ['32'], ['38'], ['39'], ['48'], ['52'], ['54'], ['55'], ['57'], ['69'], ['70'], ['71'], ['72'], ['73'], ['75'], ['78'], ['93'], ['112'], ['116'], ['120'], ['132'], ['140'], ['145'], ['151'], ['161'], ['175'], ['177'], ['183'], ['196'], ['204'], ['205'], ['214'], ['217'], ['229'], ['236'], ['239'], ['242'], ['274'], ['276'], ['279'], ['280'], ['283'], ['285'], ['296'], ['334'], ['346'], ['349'], ['350'], ['354'], ['362'], ['368'], ['381'], ['387'], ['390'], ['392'], ['401'], ['411'], ['413'], ['419'], ['438'], ['449'], ['450'], ['460'], ['470'], ['471'], ['472'], ['477'], ['480'], ['487'], ['489'], ['494'], ['509'], ['510'], ['522'], ['523'], ['526'], ['529'], ['538'], ['541'], ['548'], ['561'], ['569'], ['571'], ['579'], ['581'], ['593'], ['597'], ['598'], ['606'], ['614'], ['617'], ['620'], ['631'], ['634'], ['638'], ['653'], ['661'], ['663'], ['674'], ['675'], ['676'], ['682'], ['684'], ['692'], ['694'], ['720'], ['722'], ['744'], ['746'], [

In [113]:
# Change the support count and report the effect of changing the minimum support count. Briefly explain the reason behind the change in resulting items.  
L = apriori(dataSet_l , 0.03)
for x in L:
  print(x)

{1: [['12'], ['32'], ['39'], ['71'], ['120'], ['145'], ['177'], ['183'], ['205'], ['217'], ['283'], ['346'], ['354'], ['362'], ['368'], ['401'], ['419'], ['438'], ['460'], ['470'], ['487'], ['489'], ['494'], ['509'], ['510'], ['529'], ['538'], ['541'], ['581'], ['598'], ['682'], ['684'], ['692'], ['720'], ['722'], ['758'], ['766'], ['775'], ['789'], ['795'], ['825'], ['829'], ['862'], ['883'], ['885'], ['888'], ['895'], ['914'], ['918'], ['919'], ['937'], ['947'], ['956'], ['966'], ['998']], 2: []}
{1: [560, 668, 690, 585, 772, 784, 731, 647, 593, 906, 692, 567, 974, 672, 1289, 594, 792, 717, 732, 627, 492, 543, 785, 513, 544, 1144, 643, 592, 484, 517, 718, 885, 816, 625, 930, 483, 945, 618, 692, 543, 502, 1048, 576, 763, 505, 576, 568, 631, 488, 598, 744, 608, 590, 612, 482], 2: []}


In [114]:
L = apriori(dataSet_l , 0.04)
for x in L:
  print(x)

{1: [['32'], ['39'], ['120'], ['145'], ['177'], ['183'], ['217'], ['283'], ['354'], ['362'], ['368'], ['419'], ['438'], ['460'], ['494'], ['529'], ['538'], ['682'], ['684'], ['692'], ['722'], ['766'], ['789'], ['829'], ['883'], ['937']], 2: []}
{1: [668, 690, 772, 784, 731, 647, 906, 692, 974, 672, 1289, 792, 717, 732, 785, 1144, 643, 718, 885, 816, 930, 945, 692, 1048, 763, 744], 2: []}


In [106]:
%%shell
jupyter nbconvert --to html '/content/dm_spring_2021_assignment2_q1.ipynb'

[NbConvertApp] Converting notebook /content/dm_spring_2021_assignment2_q1.ipynb to html
[NbConvertApp] Writing 321893 bytes to /content/dm_spring_2021_assignment2_q1.html
